# Modular CerberusTS Experimentation

First we make sure we are in the base working directory (this is probably not necessary if using the package):

In [1]:
import os
os.chdir("..")

Then we import the necessary CerberusTS modules. 

In [2]:
from cerberus_ts import Cerberus, train_cerberus
from cerberus_ts import TimeseriesDataPreparer, ResponseGenerator
from cerberus_ts import Foresight, train_foresight

import pandas as pd

c:\Users\johnm\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Here we have the ability to do some custom configuration:

In [3]:
from cerberus_ts import CerberusConfig
CerberusConfig.set_masked_norm_zero = True

## Dataset Loading

Next we need a Pandas Dataframe of some wide-formatted mutlivariate timeseries data. Here we will use the Jena Climate Data found here: https://www.kaggle.com/datasets/mnassrib/jena-climate

In [4]:
df = pd.read_csv(r"data/jena_climate_2009_2016.csv",
                parse_dates=['Date Time'], 
                index_col=['Date Time'])
df.index = pd.to_datetime(df.index, format='%d.%m.%Y %H:%M:%S')
df = df.iloc[:5000,:]

## User Argument Specification
There are a number of different arguments that can be provided to Cerberus, shown below:

In [5]:
# What ratio of data needs to be available for the various call, response, and context windows to use that timestamp?
thresholds = {
    'call': 1,
    'response': 1,
    'context_0': 1,
    'context_1': 1,
    'context_2': 1
}

# How big should the call, context(s), and response windows be?
sizes = {
    'call': 24,
    'response': 8,
    'context_0': 24,
    'context_1': 12,
    'context_2': 6
}

# What timestep should each head use?
window_timesteps = {
    'call': '10T',
    'response': '10T',
    'context_0': '1H',
    'context_1': '2H',
    'context_2': '6H'
}

# Which columns should be used for each head?
feature_indexes = {
    'call': range(0,14),
    'response': [0, 1, 4],
    'context_0': range(0,14),
    'context_1': range(0,14),
    'context_2': range(0,14)   
}

## Prepare Dataset

We will use CerberusTS's TimeseriesDataPreparer class to create Torch dataloaders for all our different heads, scaled, downsampled, and coil-normalized for easy Cerberus use. 

In [6]:
# Initialize the preparer
preparer = TimeseriesDataPreparer(df, sizes, thresholds, feature_indexes, window_timesteps, train_len = 20_000, feature_range = (0, 1), batch_size = 100)

# Prepare the data
preparer.prepare_data()

## Foresight Training

First, we can optionally train Foresight to aid CerberusTS:

In [7]:
foresight = Foresight(sizes = sizes, 
                      feature_indexes = feature_indexes, 
                      csize = 128, 
                      hsize = [128, 128], 
                      pool_size = [1,1],
                      head_layers=["conv", "conv"])

In [8]:
foresight = train_foresight(foresight, preparer.dataloaders, num_epochs = 30)

Epoch [1/30], Loss: 0.020291858737667402
Epoch [2/30], Loss: 0.008772262741501132
Epoch [3/30], Loss: 0.0069979279239972434
Epoch [4/30], Loss: 0.005685849852549533
Epoch [5/30], Loss: 0.004411490069081386
Epoch [6/30], Loss: 0.00325735920186465
Epoch [7/30], Loss: 0.002137092831544578
Epoch [8/30], Loss: 0.0011166552407667041
Epoch [9/30], Loss: 0.0005500523519003763
Epoch [10/30], Loss: 0.00026689371613125936
Epoch [11/30], Loss: 0.00012350563374639024
Epoch [12/30], Loss: 6.543229557185744e-05
Epoch [13/30], Loss: 5.610925426784282e-05
Epoch [14/30], Loss: 7.801234515985318e-05
Epoch [15/30], Loss: 9.885420011414681e-05
Epoch [16/30], Loss: 0.00016359915922900352
Epoch [17/30], Loss: 0.0007408944888932941
Epoch [18/30], Loss: 0.0006130888246116228
Epoch [19/30], Loss: 8.560887759207011e-05
Epoch [20/30], Loss: 1.1966528891207418e-05
Epoch [21/30], Loss: 7.4935284646926445e-06
Epoch [22/30], Loss: 1.1546432718508489e-05
Epoch [23/30], Loss: 2.4735335179381462e-05
Epoch [24/30], Loss:

## CerberusTS Training

With the Foresight model trained (and weights frozen), we can pass this into a Cerberus model and train the remaining weights to generate predictions. 

In [9]:
model = Cerberus(sizes=sizes, 
                 feature_indexes=feature_indexes, 
                 foresight=foresight, 
                 pool_size = [1,1],
                 head_layers=["conv","conv"])

In [10]:
model = train_cerberus(model, preparer.dataloaders, num_epochs = 60)

Epoch [1/60], Loss: 0.0012699975030651936
Epoch [2/60], Loss: 0.000993401922751218
Epoch [3/60], Loss: 0.0009452742446834843
Epoch [4/60], Loss: 0.0009428700086815903
Epoch [5/60], Loss: 0.0009346467270127808
Epoch [6/60], Loss: 0.0009129920138123756
Epoch [7/60], Loss: 0.0008594421177015951
Epoch [8/60], Loss: 0.0007924715107462058
Epoch [9/60], Loss: 0.0007048235659021884
Epoch [10/60], Loss: 0.0006360380943321312
Epoch [11/60], Loss: 0.0005818628436342503
Epoch [12/60], Loss: 0.0005413601315619114
Epoch [13/60], Loss: 0.0005191428734300038
Epoch [14/60], Loss: 0.00048580218902012954
Epoch [15/60], Loss: 0.00045885445821719864
Epoch [16/60], Loss: 0.0004493673659938698
Epoch [17/60], Loss: 0.0004241602509864606


## Results Review

CerberusTS has some built in functionality for generating responses as well as visualizing the results. 

### Normalized Response Review

In [ ]:
# Intialize Response Generator
generator = ResponseGenerator(model, preparer.sliced_data, feature_indexes, preparer.max_change_dfs)

# Generate a response for a specific index
sel_index = 2130
generator.generate_response(sel_index)

generator.plot_normalized_responses()

### Unscaled Response Review

In [ ]:
generator.plot_unscaled_responses(preparer.min_max_df, feature_indexes)